<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [22]:
##### Begin Working Here #####

#upload file from local drive
#read in CSV

import pandas as pd
pd.set_option('display.max_rows', 200)

df = pd.read_csv('LoanStats_2017Q1 2.csv', skiprows=1, skipfooter=2)
print(df.shape)
df.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """


(96779, 122)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
0,NaN,NaN,3600,3600,3600,36 months,7.49%,111.97,A,A4,Code/Compliance Inspector,10+ years,MORTGAGE,120000.0,Not Verified,Mar-2017,Issued,n,NaN,NaN,other,Other,467xx,IN,18.90,0,Aug-1992,1,NaN,96.0,18,1,5658,14.9%,33,w,3600.00,3600.00,0.00,0.00,...,3,2,0,9.0,NaN,2.0,NaN,0,1,4,2,3,17,12,16,4,18,0.0,0,0,6,100.0,0.0,1,0,93599,36506,9000,55699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,15000,15000,15000,60 months,14.99%,356.78,C,C4,Senior Superintendent,10+ years,MORTGAGE,125000.0,Not Verified,Mar-2017,Issued,n,NaN,NaN,other,Other,064xx,CT,17.25,0,Sep-1989,0,NaN,NaN,11,0,53167,75.3%,18,w,15000.00,15000.00,0.00,0.00,...,16,16,2,16.0,NaN,16.0,NaN,0,5,7,6,8,6,8,10,7,11,0.0,0,0,0,100.0,66.7,0,0,148572,69364,60300,30224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,8400,8400,8400,36 months,11.39%,276.56,B,B3,IT Tech,8 years,MORTGAGE,50000.0,Source Verified,Mar-2017,Issued,n,NaN,NaN,other,Other,454xx,OH,15.63,0,Apr-2005,0,NaN,NaN,14,0,12831,30.3%,30,w,8400.00,8400.00,0.00,0.00,...,4,4,4,4.0,NaN,10.0,NaN,0,4,5,7,11,9,11,16,5,14,0.0,0,0,5,100.0,14.3,0,0,205671,51591,36100,36876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000,36 months,10.49%,130.00,B,B2,Dental hygienist,2 years,RENT,50000.0,Source Verified,Mar-2017,Issued,n,NaN,NaN,major_purchase,Major purchase,768xx,TX,33.61,1,Apr-2003,0,6.0,37.0,10,1,4388,33.2%,33,w,4000.00,4000.00,0.00,0.00,...,15,7,6,120.0,NaN,9.0,61.0,1,1,4,1,3,15,6,12,4,10,0.0,0,0,1,87.5,100.0,0,0,152852,136208,2500,139652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,6000,6000,6000,36 months,7.24%,185.93,A,A3,Program Manager,3 years,RENT,125000.0,Source Verified,Mar-2017,Current,n,NaN,NaN,credit_card,Credit card refinancing,981xx,WA,9.25,0,Jul-2004,1,54.0,NaN,16,0,9571,41.3%,27,w,5850.27,5850.27,164.21,164.21,...,7,3,0,7.0,NaN,4.0,NaN,9,3,3,5,5,22,5,5,3,16,0.0,0,0,3,59.1,20.0,0,0,72865,60622,23200,49665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#Look to see if there are any columns that are all NaN values


df.isnull().sum().sort_values()

In [0]:
#Drop the Columns 'url', 'member_id', 'desc', 'id'

df = df.drop(['url', 'member_id', 'desc', 'id'], axis=1)
df.isnull().sum().sort_values()

In [33]:
#Clean up the term column

df['term']


0         36 months
1         60 months
2         36 months
3         36 months
4         36 months
            ...    
96774     60 months
96775     36 months
96776     36 months
96777     36 months
96778     60 months
Name: term, Length: 96779, dtype: object

In [29]:
#check how these are formatted

df['term'][0]

' 36 months'

In [0]:
term = ' 36 months'


In [31]:
term.strip()

'36 months'

In [39]:
term.strip(' months')

'36'

In [40]:
#change term from object to integer

int(term.strip().strip(' months'))

36

In [0]:
#function for changing term to integer

def term_to_int(cell_contents):

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01